# Colisões entre partículas

Terceira lei de Newton, que diz que a força de ação é igual em módulo e em sentido oposto à força de reação. Um par de forças de ação e reação é produzido pela interação entre dois objetos e pode ocorrer tanto em forças de longa distância (como a força gravitacional) como em forças de contato produzidas em uma colisão. Desta forma, seja A e B dois corpos em interação, temos que ${\bf F}_{A|B} = -{\bf F}_{B|A}$, onde usamos a notação $\bf{F}_{X|Y}$ para denotar a força em X produzida pela interação com Y.

Este tipo de interção implica em conservação do momento angular linear total, definido como ${\bf p} = m {\bf v}$. Basta calcular

$$\frac{d}{dt}{\bf P} \equiv \frac{d}{dt}\left(m_A {\bf v}_A + m_B {\bf v}_B\right) = m_A {\bf a}_A + m_B{\bf a}_B$$

Substituimos $F = ma$, de acordo com a segunda Lei e lembrando que a força em A, $\bf{F}_A = \bf{F}_{A|B}$ é devido à interação com B e vice-versa:

$$\frac{d}{dt}{\bf P} = {\bf F}_{A|B} + {\bf F}_{B|A} = 0$$

O fato da derivada do momento total se anular (em outras palavras, momento se mantêm constante) é a base do estudo de colisões. Vamos derivar algumas equações importantes e realizar simulações com o Pymunk para testar estas teorias.

Este notebook mistura cálculos manuais com alguns cálculos feitos utilizando o pacote algébrico sympy.

In [83]:
import sympy
from sympy import *

# Criamos variáveis algébricas
mA, mB, u, vA, vB = sympy.symbols('m_A,m_B,u,v_A,v_B')

# Estas variáveis são objetos especiais que constroem expresões matemáticas

Pi = mA * u             # momento total com partícula A como projétil de velocidade u e B como alvo parado
Pf = mA * vA + mB * vB  # momento total após a colisão
eq1 = Eq(Pi, Pf)        # equação que expressa conservação do momento 
eq1

Eq(m_A*u, m_A*v_A + m_B*v_B)

## Coeficiente de restituição

Note que apenas com a conservação de momeneto, não é possível resolver as duas incógnitas, $\bf{v}_A$ e $\bf{v}_B$, a partir de apenas uma equação. Precisamos de uma condição adicional para caracterizar a colisão e ela é dada pela equação da elasticidade. Nela, postulamos que a velocidade relativa de entrada é proporcional à velocidade relativa de saída, com uma constante de propocionalidade dada. O sinal da velocidade relativa se inverte após a colisão já que saímos de uma condição de aproximação para a de afastamento, assim, definimos

$${\bf v}^{rel}_{f} = -e {\bf v}^{rel}_{0},$$

a constante de proporcionalidade $e\in[0,1]$ é conhecida como coeficiente de restituição. No valor extremo $e=1$ temos a reflexão especular e quando $e=0$, temos uma colisão totalmente inelástica em que os dois corpos permanecem em contato.

No caso da colisão considerada anteriormente, ${\bf v}^{rel}_{0} = u$ e ${\bf v}^{rel}_{f} = {\bf v}_A - {\bf v}_B$.

In [84]:
e = sympy.symbols('e')

eq2 = Eq(vA - vB, - e * u)
eq2

Eq(v_A - v_B, -e*u)

Agora temos 2 equações e duas incógnitas e podemos facilmente resolver o sistema. Primeiro, isolamos $\bf{v}_A$ em uma das equações. O passo seguinte é substituir na outra para obter $\bf{v}_B$

In [85]:
vA_ans = solve(eq2, vA)[0]
Eq(vA, vA_ans)

Eq(v_A, -e*u + v_B)

In [86]:
eq3 = eq1.subs(vA, vA_ans)
eq3

Eq(m_A*u, m_A*(-e*u + v_B) + m_B*v_B)

In [88]:
vB_ans = solve(eq3, vB)[0]
vA_ans = solve(eq1.subs(vB, vB_ans), vA)[0]

display(Eq(vA, vA_ans))
display(Eq(vB, vB_ans))

Eq(v_A, u*(-e*m_B + m_A)/(m_A + m_B))

Eq(v_B, m_A*u*(e + 1)/(m_A + m_B))

## Casos especiais

Podemos agora verificar vários casos especiais:

* Colisões totalmente elásticas
    - massas iguais
    - A mais pesado que B
    - B mais pesado que A
* Colisões totalmente inelásticas
    - massa de B infinitamente maior que A
    - massa de A infinitamente maior que B

In [93]:
# e = 1, massas iguais
m = sympy.symbols('m')
subs = [(mA, m), (mB, m), (e, 1)]

display(Eq(vA, vA_ans.subs(subs)))
display(Eq(vB, vB_ans.subs(subs)))

Eq(v_A, 0)

Eq(v_B, u)

In [94]:
# e = 1, massas diferentes
m, r = sympy.symbols('m,r')
subs = [(mB, r * mA), (e, 1)]

display(Eq(r, mB / mA))
display(Eq(vA, vA_ans.subs(subs).collect(mA)))
display(Eq(vB, vB_ans.subs(subs).collect(mA)))

Eq(r, m_B/m_A)

Eq(v_A, u*(1 - r)/(r + 1))

Eq(v_B, 2*u/(r + 1))

In [95]:
# e = 0

r = sympy.symbols('r')
v_ans_r = v_ans.subs(mB, r * mA).collect(mA)
Eq(vB, v_ans_r)

Eq(v_B, u/(r + 1))

In [96]:
# massa de B infinitamente maior que A (r -> oo)

v_ans_r.limit(r, oo)

0

In [97]:
# massa de A infinitamente maior que B (r -> 0)

v_ans_r.limit(r, 0)

u